Урок 3. Парсинг данных. HTML, Beautiful Soap

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [2]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [9]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}
url_hh = 'https://spb.hh.ru/search/vacancy'

In [14]:
def get_html(url, params=''):
    response = requests.get(url, headers = headers, params = params)
    
    return response

In [15]:
def get_hh_content(response):
    soup = bs(response, 'lxml')
    items = soup.find_all('div', class_ = 'serp-item')
    
    hh_vacancy = []
    for item in items:
        hh_vacancy.append(
            {'site': 'HeadHunter',
             'title': item.find('a', {'class': 'serp-item__title'}).get_text(),
             'link': item.find('a', {'class': 'serp-item__title'})['href'],
             'salary': item.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}),
             'city': item.find('div', {'data-qa': 'vacancy-serp__vacancy-address'}).contents[0].strip(', '),
             'organization': item.find('a', {'class': 'bloko-link_kind-tertiary'}),
             'note': item.find('div', {'class': 'vacancy-serp-item__label'})})

    for i in hh_vacancy:
        try:
            i['salary'] = i['salary'].text
        except:
            i['salary'] = None
        if i['salary']:
            salary_list = i['salary'].split(' ')
            if salary_list[0] == 'от':
                i['salary_min'] = salary_list[1]
                i['salary_max'] = None
            elif salary_list[0] == 'до':
                i['salary_min'] = None
                i['salary_max'] = salary_list[1]
            else:
                i['salary_min'] = salary_list[0]
                i['salary_max'] = salary_list[2]
            i['salary_currency'] = salary_list[-1]
        else:
            i['salary_min'] = None
            i['salary_max'] = None
            i['salary_currency'] = None
        i.pop('salary')
        if i['city']:
            city_list = i['city'].split(',')
            i['city'] = city_list[0]
        if i['organization'] != None:
            i['organization'] = i['organization'].get_text()
        if i['note'] != None:
            i['note'] = i['note'].get_text()
    
    return hh_vacancy

In [23]:
def parser_hh():
    post = str(input('Введите название вакансий для парсинга: '))
    pages = int(input('Введите количество страниц для парсинга: '))
    html = get_html(url_hh)
    
    if html.status_code == 200:
        vacancy = []
        for page in range(1, pages + 1):
            print(f'HeadHunter, страница: {page}')
            html = get_html(url_hh, params = {'text': post, 'page': page})
            vacancy.extend(get_hh_content(html.text))
        hh_result = pd.DataFrame(vacancy)
    
    else:
        hh_result = html.status_code
    
    return hh_result

parser_hh()

Введите название вакансий для парсинга: Аналитик
Введите количество страниц для парсинга: 2
HeadHunter, страница: 1
HeadHunter, страница: 2


,site,title,link,city,organization,note,salary_min,salary_max,salary_currency
0,HeadHunter,Маркетолог-аналитик,https://adsrv.hh.ru/click?b=372132&c=6&meta=i8...,Москва,ООО Ваша Кадровая Служба,Можно из дома,45 000,None,руб.
1,HeadHunter,Младший бизнес-аналитик,https://spb.hh.ru/vacancy/77330688?from=vacanc...,Санкт-Петербург,ООО Бизнес Технологии,None,50 000,None,руб.
2,HeadHunter,Аналитик,https://spb.hh.ru/vacancy/77684660?from=vacanc...,Санкт-Петербург,ANCOR,Можно из дома,70 000,None,руб.
3,HeadHunter,Системный аналитик,https://spb.hh.ru/vacancy/77135735?from=vacanc...,Москва,ООО Автомакон,Можно из дома,108 000,395 000,руб.
4,HeadHunter,Аналитик,https://spb.hh.ru/vacancy/77004932?from=vacanc...,Москва,ООО Xploration Capital,None,150 000,None,руб.
5,HeadHunter,Аналитик коммерческого отдела,https://spb.hh.ru/vacancy/77441415?from=vacanc...,Москва,ООО Алмазгеобур,None,180 000,None,руб.
6,HeadHunter,Аналитик данных (Junior),https://spb.hh.ru/vacancy/76991488?from=vacanc...,Владивосток,DNS Технологии,None,70 000,None,руб.
7,HeadHunter,Стажер-аналитик,https://spb.hh.ru/vacancy/77631148?from=vacanc...,Ташкент,ООО MERIDIAN SCHOOL,None,2 500 000,None,сум
8,HeadHunter,Системный аналитик,https://spb.hh.ru/vacancy/77643387?from=vacanc...,Воронеж,SoftGamings,Откликнитесь среди первых,85 000,200 000,руб.
9,HeadHunter,Аналитик Excel в стартап,https://spb.hh.ru/vacancy/77617560?from=vacanc...,Москва,Intapplex,None,50 000,None,руб.


In [24]:
vacancy = parser_hh()

Введите название вакансий для парсинга: Аналитик
Введите количество страниц для парсинга: 2
HeadHunter, страница: 1
HeadHunter, страница: 2


In [25]:
vacancy.to_csv('vacancy.csv', index = False, encoding = 'utf8')